<a href="https://colab.research.google.com/github/kevinvbc/deeproad/blob/main/Gorilla_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gorilla Data Engineer Assignment

In [2]:
import pandas as pd
import numpy as np
import random
from itertools import product


In [3]:
# get data from repo
!wget "https://raw.githubusercontent.com/kevinvbc/gorilla/master/gorilla_test_data.xlsx"
xls = pd.ExcelFile('gorilla_test_data.xlsx')
meter_list = pd.read_excel(xls, 'meter_list')
forecast_table = pd.read_excel(xls, 'forecast_table')
rate_table = pd.read_excel(xls, 'rate_table')

--2023-03-15 21:29:47--  https://raw.githubusercontent.com/kevinvbc/gorilla/master/gorilla_test_data.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121386 (119K) [application/octet-stream]
Saving to: ‘gorilla_test_data.xlsx’

gorilla_test_data.x 100%[===================>] 118.54K  --.-KB/s    in 0.01s   

2023-03-15 21:29:47 (9.83 MB/s) - ‘gorilla_test_data.xlsx’ saved [121386/121386]



##Exploratory Data Analysis + Preprocessing


In [3]:
# EDA
forecast_table

,meter_id,date,kwh
0,14676236,2020-06-01,22.070768
1,14676236,2020-06-02,19.170720
2,14676236,2020-06-03,23.555111
3,14676236,2020-06-04,18.220712
4,14676236,2020-06-05,14.196134
...,...,...,...
3407,88357331,2022-09-27,441.014725
3408,88357331,2022-09-28,441.512055
3409,88357331,2022-09-29,437.240657
3410,88357331,2022-09-30,517.608354


In [4]:
# aq_max_kwh column seems to have NaNs
rate_table['aq_max_kwh'].unique()
rate_table[(rate_table['aq_max_kwh'].isna())]

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
2,2020-04-01,EA1,732000,NaN,0.2875
5,2020-04-01,EA2,732000,NaN,0.1849
8,2020-04-01,EA3,732000,NaN,0.2492
11,2020-04-01,EA4,732000,NaN,0.2889
14,2020-04-01,EM1,732000,NaN,0.2734
...,...,...,...,...,...
1127,2024-10-01,WA1,732000,NaN,0.8735
1130,2024-10-01,WA2,732000,NaN,0.5638
1133,2024-10-01,WM1,732000,NaN,0.6461
1136,2024-10-01,WM2,732000,NaN,0.7534


In [5]:
# fill NaNs with very large value for aq_max_kwh 
large_value = 5000000
rate_table['aq_max_kwh'] = rate_table['aq_max_kwh'].fillna(large_value)

# aq_max_kwh can be an integer, just like aq_min_kwh
rate_table['aq_max_kwh'] = rate_table['aq_max_kwh'].astype('int64')
rate_table

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200,0.2652
1,2020-04-01,EA1,73200,732000,0.1980
2,2020-04-01,EA1,732000,5000000,0.2875
3,2020-04-01,EA2,0,73200,0.2970
4,2020-04-01,EA2,73200,732000,0.1524
...,...,...,...,...,...
1135,2024-10-01,WM2,73200,732000,0.4537
1136,2024-10-01,WM2,732000,5000000,0.7534
1137,2024-10-01,WM3,0,73200,0.7263
1138,2024-10-01,WM3,73200,732000,0.6109


In [6]:
# add date on which rate finishes to rate_table
rate_table['finish_date'] = rate_table.groupby(['exit_zone','aq_min_kwh'])['date'].shift(-1)
rate_table

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh,finish_date
0,2020-04-01,EA1,0,73200,0.2652,2020-10-01
1,2020-04-01,EA1,73200,732000,0.1980,2020-10-01
2,2020-04-01,EA1,732000,5000000,0.2875,2020-10-01
3,2020-04-01,EA2,0,73200,0.2970,2020-10-01
4,2020-04-01,EA2,73200,732000,0.1524,2020-10-01
...,...,...,...,...,...,...
1135,2024-10-01,WM2,73200,732000,0.4537,NaT
1136,2024-10-01,WM2,732000,5000000,0.7534,NaT
1137,2024-10-01,WM3,0,73200,0.7263,NaT
1138,2024-10-01,WM3,73200,732000,0.6109,NaT


In [7]:
# join forecast and meter table to access exit_zone information
# set index and using .join() is optimal way of joining pandas dataframes
forecast_per_zone = forecast_table.set_index('meter_id').join(meter_list.set_index('meter_id'), on = "meter_id", how = "left")
forecast_per_zone = forecast_per_zone.sort_values(['date', 'exit_zone'])
forecast_per_zone



,date,kwh,aq_kwh,exit_zone
meter_id,,,,
14676236,2020-06-01,22.070768,28978,EA1
50264822,2020-06-01,278.759084,265667,NT1
88357331,2020-06-01,343.883152,484399,SE2
34509937,2020-06-01,78.611504,78324,SO1
14676236,2020-06-02,19.170720,28978,EA1
...,...,...,...,...
34509937,2022-09-30,81.478131,78324,SO1
14676236,2022-10-01,37.122479,28978,EA1
50264822,2022-10-01,328.192979,265667,NT1


In [10]:
def compare_annual_quantity(row):

    ### This function retrieves the applicable rate from the rate_table and returns it ###
    # Make a Boolean mask that adheres to the conditions of the correct rate
    mask =   (rate_table['date'] <= row['date']) & (rate_table['finish_date'] > row['date'])  & (rate_table['exit_zone'] == row['exit_zone']) & (rate_table['aq_min_kwh'] <= row['aq_kwh']) & (rate_table['aq_max_kwh'] > row['aq_kwh'])
    relevant_rows = rate_table.loc[mask]

    # Return the relevant rates
    if len(relevant_rows) == 0:
        return None
    else:
        #return relevant_rows['rate_p_per_kwh'].values.tolist()
        
        return relevant_rows['rate_p_per_kwh'].squeeze()
        #return relevant_rows['rate_p_per_kwh']#.to_numpy()




In [11]:
# Add the relevant rates to the forecast table for each day and for each meter
forecast_per_zone['relevant_rate_p_per_kwh'] = forecast_per_zone.apply(compare_annual_quantity, axis=1).astype('float')
forecast_per_zone

,date,kwh,aq_kwh,exit_zone,relevant_rate_p_per_kwh
meter_id,,,,,
14676236,2020-06-01,22.070768,28978,EA1,0.2652
50264822,2020-06-01,278.759084,265667,NT1,0.2181
88357331,2020-06-01,343.883152,484399,SE2,0.2318
34509937,2020-06-01,78.611504,78324,SO1,0.2743
14676236,2020-06-02,19.170720,28978,EA1,0.2652
...,...,...,...,...,...
34509937,2022-09-30,81.478131,78324,SO1,0.4316
14676236,2022-10-01,37.122479,28978,EA1,0.4674
50264822,2022-10-01,328.192979,265667,NT1,0.3844


In [12]:
# Compute daily charge
forecast_per_zone['daily_charge'] = forecast_per_zone['relevant_rate_p_per_kwh']*forecast_per_zone['kwh']
forecast_per_zone

,date,kwh,aq_kwh,exit_zone,relevant_rate_p_per_kwh,daily_charge
meter_id,,,,,,
14676236,2020-06-01,22.070768,28978,EA1,0.2652,5.853168
50264822,2020-06-01,278.759084,265667,NT1,0.2181,60.797356
88357331,2020-06-01,343.883152,484399,SE2,0.2318,79.712115
34509937,2020-06-01,78.611504,78324,SO1,0.2743,21.563136
14676236,2020-06-02,19.170720,28978,EA1,0.2652,5.084075
...,...,...,...,...,...,...
34509937,2022-09-30,81.478131,78324,SO1,0.4316,35.165961
14676236,2022-10-01,37.122479,28978,EA1,0.4674,17.351047
50264822,2022-10-01,328.192979,265667,NT1,0.3844,126.157381


In [25]:
forecast_per_zone['kwh'].dtype

dtype('float64')

In [23]:
# Total cost and consumption per meter in pounds
total_cost_per_meter =  pd.DataFrame((forecast_per_zone['daily_charge'].groupby('meter_id').sum()*0.01).round(2))
total_cost_per_meter.rename(columns={"daily_charge": "Total_cost(pounds)"},inplace=True)
total_cost_per_meter['Total Estimated Consumption(kWh)'] = (forecast_per_zone['kwh'].groupby('meter_id').sum().round(2))
total_cost_per_meter

,Total_cost(pounds),Total Estimated Consumption(kWh)
meter_id,,
14676236,100.15,28978.0
34509937,275.49,78324.0
50264822,731.24,265667.0
88357331,1433.16,484399.0


##Random Meter Generator

In [9]:
def meter_generator(size):

  random.seed(49)
  valid_exit_zones = rate_table['exit_zone'].unique().tolist()
  meter_ids = random.sample(range(10000000, 99999999), size)
  annual_quantities = random.sample(range(10000, 100000), size)
  exit_zones = random.sample(valid_exit_zones, size) #sample from list of valid exit zones 

  #meter_list = [list(x) for x in zip(meter_ids, annual_quantities, exit_zones)]

  return meter_ids, annual_quantities, exit_zones



In [41]:
meters,aq,exit_zones = meter_generator(10)


In [42]:
meters

[18977015,
 56215175,
 65464511,
 24831694,
 53418792,
 84404309,
 78623525,
 16667130,
 16013958,
 47034362]

## Mock Consumption Data Generator

In [43]:
forecast_start_date = forecast_table['date'].min()

def mock_consumption_generator(meter_list, start_date = forecast_start_date, duration = 852):
  
  datelist = pd.date_range(forecast_start_date, periods=duration).tolist()
  df = pd.DataFrame(list(product(meter_list, datelist)), columns=['meter_id', 'date'])
  size = len(meter_list)*duration
  integer_list = random.sample(range(100000, 10000000), size)
  # create random floats for forecast per day
  df['kWh'] = [round(x/100000,5) for x in integer_list]


  return df



In [45]:
mock_data = mock_consumption_generator(meters)
mock_data

,meter_id,date,kWh
0,18977015,2020-06-01,25.86110
1,18977015,2020-06-02,88.31439
2,18977015,2020-06-03,28.06001
3,18977015,2020-06-04,91.33446
4,18977015,2020-06-05,41.48129
...,...,...,...
8515,47034362,2022-09-26,12.19682
8516,47034362,2022-09-27,4.39432
8517,47034362,2022-09-28,87.42353
8518,47034362,2022-09-29,47.58817


In [ ]:
#rate_table.set_index('date').resample('D')

#df = df.drop(['type', 'unique_id'], axis=1)
#df.beginning_time = pd.to_datetime(df.beginning_time)
#df.end_time = pd.to_datetime(df.end_time)
#rate_table = rate_table.melt(rate_table, id_vars=['exit_zone'], value_name='rate_p_per_kwh').drop('variable', axis=1)
#rate_table.set_index('rate_p_per_kwh', inplace=True)



exit_zone  aq_min_kwh  aq_max_kwh  rate_p_per_kwh
exit_zone date                                                        
EA1       2020-04-01       EA1           0       73200          0.2652
          2020-04-02       EA1           0       73200          0.2652
          2020-04-03       EA1           0       73200          0.2652
          2020-04-04       EA1           0       73200          0.2652
          2020-04-05       EA1           0       73200          0.2652
...                        ...         ...         ...             ...
WM3       2024-09-27       WM3           0       73200          0.6485
          2024-09-28       WM3           0       73200          0.6485
          2024-09-29       WM3           0       73200          0.6485
          2024-09-30       WM3           0       73200          0.6485
          2024-10-01       WM3           0       73200          0.7263

[62510 rows x 4 columns]

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/google/colab/data_table.py", line 187, in _repr_mimebundle_
    dataframe = self._preprocess_dataframe()
  File "/usr/local/lib/python3.9/dist-packages/google/colab/data_table.py", line 175, in _preprocess_dataframe
    dataframe = dataframe.reset_index()
  File "/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py", line 5848, in reset_index
    new_obj.insert(0, name, level_values)
  File "/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py", line 4443, in insert
    raise ValueError(f"cannot insert {column}, already exists")
ValueError: cannot insert exit_zone, already exists
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/google/colab/data_table.py", line 199, in _repr_javascript_module_
    return self._gen_js(self._preproce

In [ ]:
rate_table

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200,0.2652
1,2020-04-01,EA1,73200,732000,0.1980
2,2020-04-01,EA1,732000,5000000,0.2875
3,2020-04-01,EA2,0,73200,0.2970
4,2020-04-01,EA2,73200,732000,0.1524
...,...,...,...,...,...
1135,2024-10-01,WM2,73200,732000,0.4537
1136,2024-10-01,WM2,732000,5000000,0.7534
1137,2024-10-01,WM3,0,73200,0.7263
1138,2024-10-01,WM3,73200,732000,0.6109


In [ ]:
rate_table['days'] = (last_date - rate_table.date.min()).days
rate_table['dates_between'] = rate_table.apply(lambda row: [row['date'] + pd.Timedelta(days=d) for d in range(row['days']+1)], axis=1)
del rate_table['days']
rate_table.explode('dates_between')

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh,dates_between
0,2020-04-01,EA1,0,73200,0.2652,2020-04-01
0,2020-04-01,EA1,0,73200,0.2652,2020-04-02
0,2020-04-01,EA1,0,73200,0.2652,2020-04-03
0,2020-04-01,EA1,0,73200,0.2652,2020-04-04
0,2020-04-01,EA1,0,73200,0.2652,2020-04-05
...,...,...,...,...,...,...
1139,2024-10-01,WM3,732000,5000000,0.4928,2027-03-29
1139,2024-10-01,WM3,732000,5000000,0.4928,2027-03-30
1139,2024-10-01,WM3,732000,5000000,0.4928,2027-03-31
1139,2024-10-01,WM3,732000,5000000,0.4928,2027-04-01


In [ ]:
rate_table

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh,days,dates_between
0,2020-04-01,EA1,0,73200,0.2652,913,"[2020-04-01 00:00:00, 2020-04-02 00:00:00, 202..."
1,2020-04-01,EA1,73200,732000,0.1980,913,"[2020-04-01 00:00:00, 2020-04-02 00:00:00, 202..."
2,2020-04-01,EA1,732000,5000000,0.2875,913,"[2020-04-01 00:00:00, 2020-04-02 00:00:00, 202..."
3,2020-04-01,EA2,0,73200,0.2970,913,"[2020-04-01 00:00:00, 2020-04-02 00:00:00, 202..."
4,2020-04-01,EA2,73200,732000,0.1524,913,"[2020-04-01 00:00:00, 2020-04-02 00:00:00, 202..."
...,...,...,...,...,...,...,...
1135,2024-10-01,WM2,73200,732000,0.4537,913,"[2024-10-01 00:00:00, 2024-10-02 00:00:00, 202..."
1136,2024-10-01,WM2,732000,5000000,0.7534,913,"[2024-10-01 00:00:00, 2024-10-02 00:00:00, 202..."
1137,2024-10-01,WM3,0,73200,0.7263,913,"[2024-10-01 00:00:00, 2024-10-02 00:00:00, 202..."
1138,2024-10-01,WM3,73200,732000,0.6109,913,"[2024-10-01 00:00:00, 2024-10-02 00:00:00, 202..."


In [ ]:

rate_table['days'] = (rate_table['end_date'] - df['start_date']).days
df['dates_between'] = df.apply(lambda row: [row['start_date'] + timedelta(days=d) for d in range(row['days']+1)], axis=1)
del df['days']
df.explode('dates_between')

In [ ]:

import datetime as dt
last_date = pd.to_datetime(max(forecast_table['date'])).floor('D')

rate_table['days'] = (pd.to_datetime(max(forecast_table['date'])).floor('D') - rate_table.date.min()).astype('timedelta64[D]').astype('int')

#(df.end_date - df.start_date).astype('timedelta64[D]').astype('int')+1
rate_table['dates_between'] = rate_table.apply(lambda row: [row['date'] + pd.timedelta(days=d) for d in range(row['days']+1)], axis=1)
del rate_table['days']
rate_table.explode('dates_between')
rate_table

AttributeError: ignored

In [ ]:
last_date = pd.to_datetime(max(forecast_table['date'])).floor('D')
last_date

Timestamp('2022-10-01 00:00:00')

In [ ]:
dates = pd.date_range(rate_table.date.min(), last_date, name='date')
d = rate_table.set_index('date').sort_index().reindex(dates,method='ffill')
d

ValueError: ignored

In [ ]:
last_date = pd.to_datetime(max(forecast_table['date'])).floor('D')

def f(df):
    dates = pd.date_range(df.date.min(), last_date, name='date')
    d = df.set_index('date').sort_index().reindex(dates, method='ffill')
    return d.reset_index().reindex_axis(df.columns, 1)

rate_table.apply(f)
rate_table

AttributeError: ignored

In [ ]:
rate_table['date'].unique()

array(['2020-04-01T00:00:00.000000000', '2020-10-01T00:00:00.000000000',
       '2021-04-01T00:00:00.000000000', '2021-10-01T00:00:00.000000000',
       '2022-04-01T00:00:00.000000000', '2022-10-01T00:00:00.000000000',
       '2023-04-01T00:00:00.000000000', '2023-10-01T00:00:00.000000000',
       '2024-04-01T00:00:00.000000000', '2024-10-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [ ]:
forecast_per_zone

,date,kwh,aq_kwh,exit_zone,relevant_rate_p_per_kwh
meter_id,,,,,
14676236,2020-06-01,22.070768,28978,EA1,NaN
14676236,2020-06-02,19.170720,28978,EA1,NaN
14676236,2020-06-03,23.555111,28978,EA1,NaN
14676236,2020-06-04,18.220712,28978,EA1,NaN
14676236,2020-06-05,14.196134,28978,EA1,NaN
...,...,...,...,...,...
88357331,2022-09-27,441.014725,484399,SE2,NaN
88357331,2022-09-28,441.512055,484399,SE2,NaN
88357331,2022-09-29,437.240657,484399,SE2,NaN


In [ ]:
forecast_per_zone['relevant_rate_p_per_kwh'].dtype

dtype('float64')

In [ ]:
forecast_per_zone['relevant_rate_p_per_kwh'].ffil(axis=0)

AttributeError: ignored

In [ ]:
forecast_per_zone

NameError: ignored

In [ ]:
forecast_per_zone

,date,kwh,aq_kwh,exit_zone,relevant_rate
meter_id,,,,,
14676236,2020-06-01,22.070768,28978,EA1,None
14676236,2020-06-02,19.170720,28978,EA1,None
14676236,2020-06-03,23.555111,28978,EA1,None
14676236,2020-06-04,18.220712,28978,EA1,None
14676236,2020-06-05,14.196134,28978,EA1,None
...,...,...,...,...,...
88357331,2022-09-27,441.014725,484399,SE2,None
88357331,2022-09-28,441.512055,484399,SE2,None
88357331,2022-09-29,437.240657,484399,SE2,None


In [ ]:
rate_table['date'] = rate_table['date'].astype('datetime64[ns]')
rate_table['aq_max_kwh'] = rate_table['aq_max_kwh'].astype('int64')
forecast_table['date'] = forecast_table['date'].astype('datetime64[ns]')
rate_table['exit_zone'].astype('string')

IntCastingNaNError: ignored

In [ ]:
daily_charge_table = forecast_table.set_index('date').join(rate_table.set_index('date'), how = "left")



In [ ]:
daily_charge_table

,meter_id,kwh,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
date,,,,,,
2020-04-01,NaN,NaN,EA1,0,73200.0,0.2652
2020-04-01,NaN,NaN,EA1,73200,732000.0,0.1980
2020-04-01,NaN,NaN,EA1,732000,NaN,0.2875
2020-04-01,NaN,NaN,EA2,0,73200.0,0.2970
2020-04-01,NaN,NaN,EA2,73200,732000.0,0.1524
...,...,...,...,...,...,...
2024-10-01,NaN,NaN,WM2,73200,732000.0,0.4537
2024-10-01,NaN,NaN,WM2,732000,NaN,0.7534
2024-10-01,NaN,NaN,WM3,0,73200.0,0.7263


In [ ]:
forecast_table.dtypes

meter_id             int64
date        datetime64[ns]
kwh                float64
dtype: object

In [ ]:
rate_table.dtypes

date              datetime64[ns]
exit_zone                 object
aq_min_kwh                 int64
aq_max_kwh               float64
rate_p_per_kwh           float64
dtype: object

In [ ]:
# ... Vectorized operation:
df["ratio"] = 100 * (df["x"] / df["y"])

# ... Non-vectorized operation:
def calc_ratio(row):
    return 100 * (row["x"] / row["y"])

In [ ]:
meter_list